In [ ]:
library(ggplot2)
library(ggh4x)
library(tidyverse)
library("patchwork")
library(enrichplot)
library(dplyr)
library(ggrepel)
library(org.Hs.eg.db)
library(clusterProfiler)
library(RColorBrewer)
library(gridExtra)
library(pheatmap)

# extended_data_fig4a
# Metadata of integrated data sets across brain regions. 

In [ ]:
library(sceasy)
library(reticulate)
use_condaenv('sceasy')
sceasy::convertFormat("Fig4a.h5", from="anndata", to="seurat",
                      outFile='fig4a.rds')

fig4<-readRDS("fig4a.rds")
#+scale_fill_manual(values = c("#0e9c9d", "#dda129"))
# correct "Oligodendrocyte(PCDH9+)" as "Microglia(PCDH9high)"
fig4$cell2 <- factor(fig4$cell2)  

# Check and add new levels to factor columns
if (!("Microglia(PCDH9high)" %in% levels(fig4$cell2))) {
  levels(fig4$cell2) <- c(levels(fig4$cell2), "Microglia(PCDH9high)")
}

# substitue values
fig4$cell2[fig4$cell2 == "Oligodendrocyte(PCDH9+)"] <- "Microglia(PCDH9high)"

meta_data<-fig4@meta.data
meta_data$type<-meta_data$study
meta<-read.csv("fig4_metadata.csv",header=T,row.names=1)
meta_data<-meta
meta_data$age <- as.numeric(gsub("\\D", "", meta_data$donor_age))
meta_data$range <- ifelse(meta_data$age > 20 & meta_data$age <= 40, "21-40",
                   ifelse(meta_data$age > 40 & meta_data$age <= 60, "41-60",
                          ifelse(meta_data$age > 60 & meta_data$age <= 80, "61-80",
                                 ifelse(meta_data$age > 80, ">80", NA))))

# Metadata cleansing 
# When the ge content of meta_data satisfies "greater than 20 and less than or equal to 40", fill in "21-40".
# If "greater than or equal to 41 and less than or equal to 60" is satisfied, fill in "41-60", 
# if "greater than or equal to 41 and less than or equal to 60" is satisfied, fill in "41-60", 
# if "more than 60 and less than or equal to 80" is satisfied, fill in Enter "61-80", 
# if "greater than 80" is satisfied, fill in ">80"

meta_data$cell2<- ordered(meta_data$cell2, levels = c("Oligodendrocyte precursor","Oligodendrocyte","Newly formed oligodendrocytes","Astrocyte","Microglia", "Microglia(PCDH9high)", "Vascular leptomeningeal cell","Fibroblast", "Endothelial","GABAergic","Glutamatergic","Glutamatergic(Pyr)"))

use_colors <- c(
  'Prefrontal cortex' = "#B3128A",
  'Hippocampus' = "#74CAFF",
  'Microglia' = "#F0E68C",
  'Microglia(PCDH9high)' = "#B1E71A",
  'Astrocyte'="#AFE1AF",
    'Endothelial' = "#50C878", 
   'Fibroblast' = "#D3B48C", 
   'GABAergic' = '#E30D5C',
       `Glutamatergic`="#CC5500", 
       `Glutamatergic(Pyr)`="#FA6055", 
       `Newly formed oligodendrocytes`="#74CAFF",
        `Oligodendrocyte`="#5C89CC",
       `Oligodendrocyte precursor`="#016091",
       `Vascular leptomeningeal cell`="#8B8002")

p1 <- ggplot(meta_data, aes(
    y = region
)) +
    geom_bar(aes(fill = cell2), position = "fill") +
   scale_fill_manual(values =use_colors)  +
    ggnewscale::new_scale_fill() +
    geom_col(data = distinct(meta_data, region, range,  study), aes(x = -.3, fill = region), width = .9) +
    geom_col(data = distinct(meta_data, region, range, study), aes(x = -.15), fill = "white", color = "white") +
    geom_text(data = distinct(meta_data,region, range,  study), aes(x = -.075, label = region))  +
    scale_x_continuous(expand = c(0, 0)) +
    facet_nested(study + range~.,
        scale = "free_y", space = "free_y",
        strip = strip_nested(
            by_layer_y = TRUE,
            background_y = list(element_part_rect(side = "l", fill = NA, colour = "black", linewidth = 1),
                                element_rect(colour = NA, fill = NA))
    )
)

p2=p1+ scale_fill_manual(values = c(
        `Prefrontal cortex` = "#B3128A",
        `Hippocampus` = "#74CAFF"))
p2
ggsave(filename="fig4-stacked-meta.pdf",plot=p2, width = 14, height = 7)

fig4$study <- factor(fig4$study)

if (!("Lau et al." %in% levels(fig4$study))) {
  levels(fig4$study) <- c(levels(fig4$study), "Lau et al.")
}
fig4$study[fig4$study == "human_brain_Lau_2020_10x"] <- "Lau et al."

fig4$study <- factor(fig4$study)

if (!("Kihara et al." %in% levels(fig4$study))) {
  levels(fig4$study) <- c(levels(fig4$study), "Kihara et al.")
}
fig4$study[fig4$study == "human_brain_PFC_Kihara_2022_10x"] <- "Kihara et al."


if (!("Ayhan et al." %in% levels(fig4$study))) {
  levels(fig4$study) <- c(levels(fig4$study), "Ayhan et al.")
}
fig4$study[fig4$study == "human_brain_source_Fatma_2020"] <- "Ayhan et al."

 
if (!("Franjic et al." %in% levels(fig4$study))) {
  levels(fig4$study) <- c(levels(fig4$study), "Franjic et al.")
}
fig4$study[fig4$study == "human_brain_hippocampus_Franjic_2021_10x"] <- "Franjic et al."


fig4_counts <- FetchData(fig4, vars = c("region", "cell2","study","donor_status","age")) %>%  
  mutate(region = factor(region, levels = c("Hippocampus","Prefrontal cortex")))


  fig4_counts <- FetchData(fig4, vars = c("region", "cell2","study","donor_status","age")) %>%  
    mutate(
        region = factor(region, levels = c("Hippocampus", "Prefrontal cortex"), exclude = NULL)
    )


### cell type proportion and signature genes
fig4 <- readRDS("fig4.rds")
fig4@meta.data$cell2[fig4@meta.data$cell2 == 'Oligodendrocyte(PCDH9+)'] <- 'Microglia(PCDH9high)'

use_colors <- c(
  `Prefrontal cortex` = "#B3128A",
  `Hippocampus` = "#74CAFF",
  `Microglia` = "#F0E68C",
  `Microglia(PCDH9high)` = "#B1E71A",
  `Astrocyte`="#AFE1AF",
   `Endothelial`="#50C878", 
   `Fibroblast`="#D3B48C", 
   `GABAergic`="#E30D5C",
       `Glutamatergic`="#CC5500", 
       `Glutamatergic(Pyr)`="#FA6055", 
       `Newly formed oligodendrocytes`="#74CAFF",
        `Oligodendrocyte`="#5C89CC",
       `Oligodendrocyte precursor`="#016091",
       `Vascular leptomeningeal cell`="#8B8002")

fig4@meta.data$cell_type<- ordered(fig4@meta.data$cell2, levels = c("Oligodendrocyte precursor","Oligodendrocyte","Newly formed oligodendrocytes","Astrocyte","Microglia", "Microglia(PCDH9high)", "Vascular leptomeningeal cell","Fibroblast", "Endothelial","GABAergic","Glutamatergic","Glutamatergic(Pyr)"))

fig4@meta.data$region<- ordered(fig4@meta.data$region, levels = c("Hippocampus","Prefrontal cortex"))

fig4_counts <- FetchData(fig4, vars = c("region", "cell_type")) %>%  
  mutate(region = factor(region, levels = c("Hippocampus","Prefrontal cortex")))


 # cell proportion 
fig4_counts_tbl <- fig4_counts %>%
  dplyr::count(cell_type, region)
write_csv(fig4_counts_tbl, path = "fig4_counts_tbl.csv")

ggplot(data = fig4_counts,aes(x = region, fill = cell_type)) +
  geom_bar(position = "fill") +
  scale_fill_manual(values = use_colors) +
  coord_flip() +
  scale_y_reverse()
ggsave2("barplot_fig4.pdf", path = "/home/hlf/hpc/jupyter_dir/BCA_code", width = 8, height = 2)


# extended_data_fig4b
The UMAP plot illustrates a subset highlighted in red circles within panel a, comprising microglia and microglia (PCDH9high) cells.

In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import anndata
from matplotlib import rcParams
sc.set_figure_params(color_map='viridis')
sc.settings.verbosity = 2

In [ ]:
%pylab inline

import warnings
warnings.filterwarnings("ignore")

#from SCCAF import *
# import scvi
from gssnng import score_cells
from glob import iglob
from tqdm import tqdm

In [ ]:
fig4b=sc.read("Fig4b.h5")
#MG-State.txt is a list of differential genes from Sun et al.(https://doi.org/10.1016/j.cell.2023.08.037)
score_cells.with_gene_sets(adata=fig4b,                         # AnnData object
                            gene_set_file='MG-State.txt', # File path of gene sets
                            groupby='Celltype',                  # Will sample neighbors within this group
                            smooth_mode='connectivity',         # Smooths matrix using distance weights from NN graph.
                            recompute_neighbors=32,             # Rebuild nearest neighbor graph with groups, 0 turns off function
                            score_method='singscore',           # Method of scoring
                            method_params={'normalization':'theoretical'},  # Special parameters for some methods 
                            ranked=True,                        # Use ranked data, True or False
                            cores=36)
#MG0:homeostatic
#MG1:neuronal surveillance
#MG3:ribosome biogenesis
#MG4:lipid homeostasis 
#MG5:phagocytic
#MG6:stress signature
#MG7:glycolytic
#MG8:inflammatory II
#MG10:inflammatory III
#MG11:antiviral
#MG12:cycling
sc.pl.matrixplot(fig4b,['MG0', 'MG1', 'MG2', 'MG3', 'MG4', 'MG5', 'MG6', 'MG7', 'MG8', 'MG10', 'MG11', 'MG12'],
                 groupby='Celltype',save='extended_data_fig4b.pdf')

# extended_data_fig4c
The donor status statistics of microglia and microglia (PCDH9high) clusters.

In [ ]:
fig4b=sc.read("Fig4b.h5")
cell_metadata = fig4b.obs
cell_metadata
cell_metadata.to_csv("fig4b_metadata.csv", index=True)

In [ ]:
meta_data<-read.csv("fig4b_metadata.csv",header=T,row.names=1)
library(ggplot2)
library(cowplot)
theme_set(theme_cowplot())

# meta_data adds a range column. 
# When the ge content of meta_data satisfies "greater than 20 and less than or equal to 40", fill in "21-40".
# If "greater than or equal to 41 and less than or equal to 60" is satisfied, fill in "41-60", 
# if "greater than or equal to 41 and less than or equal to 60" is satisfied, fill in "41-60", if "more than 60 and less than or equal to 80" is satisfied, fill in Enter "61-80", if "greater than 80" is satisfied, fill in ">80"
meta_data$donor_status<- ordered(meta_data$donor_status, levels = c("Healthy","Temporal lobe epilepsy","Alzheimer's disease","Relapsing-remitting multiple sclerosis","Secondary progressive multiple sclerosis"))
use_colors <- c(
  `Prefrontal cortex` = "#B3128A",
  `Hippocampus` = "#74CAFF",
  `Microglia` = "#F0E68C",
  `Microglia(PCDH9high)` = "#B1E71A",
  `Healthy` = "#E68700",
  `Temporal lobe epilepsy`  ="#9667BF",
                 `Alzheimer's disease`="#5D6AB3",
                 `Relapsing-remitting multiple sclerosis`="#52BEA5",
                 `Secondary progressive multiple sclerosis`="#9BCA44")

meta_data$Celltype<- ordered(meta_data$Celltype, levels = c("Microglia","Microglia(PCDH9high)"))

use_colors <- c( `Healthy` = "#E68700",
                 `Alzheimer's disease`="#5D6AB3",
                 `Temporal lobe epilepsy`  ="#9667BF",
                 `Relapsing-remitting multiple sclerosis`="#52BEA5",
                 `Secondary progressive multiple sclerosis`="#9BCA44")
p1<-ggplot(data = meta_data, aes(x = Celltype, fill = donor_status)) +
    geom_bar(position = "fill") +
    scale_fill_manual(values = use_colors) +
    coord_flip() +
    scale_y_reverse()

use_colors3 <- c(`Prefrontal cortex` = "#B3128A",
  `Hippocampus` = "#74CAFF")

p3<-ggplot(data = meta_data, aes(x = region, fill = donor_status)) +
  geom_bar(position = "fill") +
  scale_fill_manual(values = use_colors) +
  coord_flip() +
  scale_y_reverse()

library(patchwork)

combined_plot <- wrap_plots(p3 + theme(legend.position = "right"),p1 + theme(legend.position = "right"),ncol = 1)
p4<-combined_plot + plot_layout(guides = 'collect')
p4

ggsave(filename="extended_data_fig4c.pdf",plot=p4, width = 8, height = 4)

# extended_data_fig4d
# Cell proportion of microglia and microglia (PCDH9high) clusters at different ages

In [ ]:
meta_data<-read.csv("fig4b_metadata.csv",header=T,row.names=1)
meta_data$age <- as.numeric(gsub("\\D", "", meta_data$donor_age))
meta_data$range <- ifelse(meta_data$age > 20 & meta_data$age <= 40, "21-40",
                   ifelse(meta_data$age > 40 & meta_data$age <= 60, "41-60",
                          ifelse(meta_data$age > 60 & meta_data$age <= 80, "61-80",
                                 ifelse(meta_data$age > 80, ">80", NA))))


use_colors <- c( `21-40` = "#DC4D4F",
                 `41-60`="#A884C9",
                 `61-80`="#FBABA9",
                 `>80`= "#468FC1")

meta_data$range<- ordered(meta_data$range, levels = c("21-40","41-60","61-80",">80"))

p1 <-ggplot(data = meta_data, aes(x = Celltype, fill = range)) +
   geom_bar(position = "fill") +
  scale_fill_manual(values = use_colors) +
    coord_flip() +scale_y_reverse()
p1


p2 <-ggplot(data = meta_data, aes(x = region, fill = range)) +
   geom_bar(position = "fill") +
  scale_fill_manual(values = use_colors) +
    coord_flip() +scale_y_reverse()
p2
library(cowplot)
library("patchwork")
combined_plot <- wrap_plots(p2 + theme(legend.position = "right"),p1 + theme(legend.position = "right"),ncol = 1)
p3=combined_plot + plot_layout(guides = 'collect')
p3

ggsave(filename="extended_data_fig4d.pdf",plot=p3, width = 8, height = 4)

# extended_data_fig4e
# Gene set scoring analysis of predicted biological function

In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import anndata
from matplotlib import rcParams
sc.set_figure_params(color_map='viridis')
sc.settings.verbosity = 2
%pylab inline

import warnings
warnings.filterwarnings("ignore")

#from SCCAF import *
# import scvi
from gssnng import score_cells
from glob import iglob
from tqdm import tqdm
fig4b=sc.read("Fig4b.h5")
#All gmt files are download from MSigDB
#MICROGLIAL_CELL_ACTIVATION
score_cells.with_gene_sets(adata=fig4b,                      
                            gene_set_file='GOBP_REGULATION_OF_MICROGLIAL_CELL_ACTIVATION.v2023.2.Hs.gmt',
                            groupby='Celltype',                
                            smooth_mode='connectivity',        
                            recompute_neighbors=32,             
                            score_method='singscore',           
                            method_params={'normalization':'theoretical'},
                            ranked=True,                       
                            cores=48)                            

#IMMUNE_RESPONSE
score_cells.with_gene_sets(adata=fig4b,                      
                            gene_set_file='GOBP_MICROGLIAL_CELL_ACTIVATION_INVOLVED_IN_IMMUNE_RESPONSE.v2023.2.Hs.gmt',
                            groupby='Celltype',                
                            smooth_mode='connectivity',        
                            recompute_neighbors=32,             
                            score_method='singscore',           
                            method_params={'normalization':'theoretical'},  
                            ranked=True,                       
                            cores=48) 
#LYSOSOME
score_cells.with_gene_sets(adata=fig4b,                      
                            gene_set_file='LYSOSOME.v2023.2.Hs.gmt',
                            groupby='Celltype',                
                            smooth_mode='connectivity',        
                            recompute_neighbors=32,             
                            score_method='singscore',           
                            method_params={'normalization':'theoretical'},  
                            ranked=True,                       
                            cores=48) 
#PHAGOCYTOSIS
score_cells.with_gene_sets(adata=fig4b,                      
                            gene_set_file='GOBP_PHAGOCYTOSIS.v2023.2.Hs.gmt',
                            groupby='Celltype',                
                            smooth_mode='connectivity',        
                            recompute_neighbors=32,             
                            score_method='singscore',           
                            method_params={'normalization':'theoretical'},  
                            ranked=True,                       
                            cores=48) 

#GLYCOLYSIS_GLUCONEOGENESIS
score_cells.with_gene_sets(adata=fig4b,                      
                            gene_set_file='KEGG_GLYCOLYSIS_GLUCONEOGENESIS.v2023.2.Hs.gmt',
                            groupby='Celltype',                
                            smooth_mode='connectivity',        
                            recompute_neighbors=32,             
                            score_method='singscore',           
                            method_params={'normalization':'theoretical'},  
                            ranked=True,                       
                            cores=48) 
#NEUROINFLAMMATORY_RESPONSE
score_cells.with_gene_sets(adata=fig4b,                      
                            gene_set_file='GOBP_NEUROINFLAMMATORY_RESPONSE.v2023.2.Hs.gmt',
                            groupby='Celltype',                
                            smooth_mode='connectivity',        
                            recompute_neighbors=32,             
                            score_method='singscore',           
                            method_params={'normalization':'theoretical'},  
                            ranked=True,                       
                            cores=48) 
#INFLAMMATORY_RESPONSE
score_cells.with_gene_sets(adata=fig4b,                      
                            gene_set_file='GOBP_INFLAMMATORY_RESPONSE.v2023.2.Hs.gmt',
                            groupby='Celltype',                
                            smooth_mode='connectivity',        
                            recompute_neighbors=32,             
                            score_method='singscore',           
                            method_params={'normalization':'theoretical'},  
                            ranked=True,                       
                            cores=48) 
#CYTOKINE_PRODUCTION
score_cells.with_gene_sets(adata=fig4b,                      
                            gene_set_file='GOBP_CYTOKINE_PRODUCTION_INVOLVED_IN_INFLAMMATORY_RESPONSE.v2023.2.Hs.gmt',
                            groupby='Celltype',                
                            smooth_mode='connectivity',        
                            recompute_neighbors=32,             
                            score_method='singscore',           
                            method_params={'normalization':'theoretical'},  
                            ranked=True,                       
                            cores=48) 
#NFKB_PATHWAY
score_cells.with_gene_sets(adata=fig4b,                      
                            gene_set_file='BIOCARTA_NFKB_PATHWAY.v2023.2.Hs.gmt',
                            groupby='Celltype',                
                            smooth_mode='connectivity',        
                            recompute_neighbors=32,             
                            score_method='singscore',           
                            method_params={'normalization':'theoretical'},  
                            ranked=True,                       
                            cores=48) 
#INTERFERON_ALPHA_PRODUCTION
score_cells.with_gene_sets(adata=fig4b,                      
                            gene_set_file='GOBP_INTERFERON_ALPHA_PRODUCTION.v2023.2.Hs.gmt',
                            groupby='Celltype',                
                            smooth_mode='connectivity',        
                            recompute_neighbors=32,             
                            score_method='singscore',           
                            method_params={'normalization':'theoretical'},  
                            ranked=True,                       
                            cores=48) 
#INTERFERON_BETA_PRODUCTION
score_cells.with_gene_sets(adata=fig4b,                      
                            gene_set_file='GOBP_INTERFERON_BETA_PRODUCTION.v2023.2.Hs.gmt',
                            groupby='Celltype',                
                            smooth_mode='connectivity',        
                            recompute_neighbors=32,             
                            score_method='singscore',           
                            method_params={'normalization':'theoretical'},  
                            ranked=True,                       
                            cores=48) 
#CELLULAR_RESPONSE_TO_INTERLEUKIN_1
score_cells.with_gene_sets(adata=fig4b,                      
                            gene_set_file='GOBP_CELLULAR_RESPONSE_TO_INTERLEUKIN_1.v2023.2.Hs.gmt',
                            groupby='Celltype',                
                            smooth_mode='connectivity',        
                            recompute_neighbors=32,             
                            score_method='singscore',           
                            method_params={'normalization':'theoretical'},  
                            ranked=True,                       
                            cores=48) 
#INFLAMMASOME_MEDIATED_SIGNALING_PATHWAY
score_cells.with_gene_sets(adata=fig4b,                      
                            gene_set_file='GOBP_INFLAMMASOME_MEDIATED_SIGNALING_PATHWAY.v2023.2.Hs.gmt',
                            groupby='Celltype',                
                            smooth_mode='connectivity',        
                            recompute_neighbors=32,             
                            score_method='singscore',           
                            method_params={'normalization':'theoretical'},  
                            ranked=True,                       
                            cores=48) 

bp=['GOBP_REGULATION_OF_MICROGLIAL_CELL_ACTIVATION', 'GOBP_MICROGLIAL_CELL_ACTIVATION_INVOLVED_IN_IMMUNE_RESPONSE', 'LYSOSOME', 'GOBP_PHAGOCYTOSIS', 'KEGG_GLYCOLYSIS_GLUCONEOGENESIS', 'GOBP_NEUROINFLAMMATORY_RESPONSE', 'GOBP_INFLAMMATORY_RESPONSE', 'GOBP_CYTOKINE_PRODUCTION_INVOLVED_IN_INFLAMMATORY_RESPONSE', 'BIOCARTA_NFKB_PATHWAY', 'GOBP_INTERFERON_ALPHA_PRODUCTION', 'GOBP_INTERFERON_BETA_PRODUCTION', 'GOBP_CELLULAR_RESPONSE_TO_INTERLEUKIN_1', 'GOBP_INFLAMMASOME_MEDIATED_SIGNALING_PATHWAY']
sc.pl.matrixplot(fig4b,bp, cmap='RdBu_r',groupby='Celltype',save="extended_data_fig4e.pdf")

# extended_data_fig4f

In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import anndata
from matplotlib import rcParams

In [ ]:
%pylab inline
import warnings
warnings.filterwarnings("ignore")
from glob import iglob
from tqdm import tqdm
sc.set_figure_params(color_map='viridis')
sc.settings.verbosity = 2

In [ ]:
%matplotlib inline

In [ ]:
fig4b=sc.read("Fig4b.h5")
top0=["ATM", "SYT11","LRP2", "LIPA", "RAB7A", "ABCA2","USP4","SRGN","RAB31","GSN","RAC1","CDC42","ELMO1","TLR2","ABL1","PTK2","CCL3","PLCG2","TNFRSF1B","APP","FOXP1","IL1RAP","GPRC5B", "SERPINE1","JAM3","IRAK2","REL","PXK","LIPA","ZEB2","ST18","IRAK2","IRAK3","PLCB1","VRK2"]
sc.pl.heatmap(fig4b, var_names=top0
              ,groupby='Celltype',vmin=-2, vmax=2,layer='scaled', cmap='RdBu_r',
              show_gene_labels=True,save="extended_data_fig4f.pdf")

# extended_data_fig4g

In [ ]:
from adpbulk import ADPBulk

fig4b=sc.read("Fig4b.h5")

# initialize the object
adpb = ADPBulk(fig4b, ["Celltype", "region"])

# perform the pseudobulking
pseudobulk_matrix = adpb.fit_transform()

# retrieve the sample meta data (useful for easy incorporation with edgeR)
sample_meta = adpb.get_meta()

print(sample_meta)
print(pseudobulk_matrix)
import pandas as pd

transposed_matrix = pseudobulk_matrix.transpose()
print(transposed_matrix)
#generate pseudobulk matrix
transposed_matrix.to_csv('mg_cell_region_transposed_pseudobulk_matrix.csv', index=True)  

In [ ]:
# Keren-Shau defined DAM genes（10.1016/j.cell.2017.05.018）
library(pheatmap)
library(dplyr)
library(RColorBrewer)
rm(list=ls())
gc()
pseudobulk_matrix <- read.table("dam-SUP-CHOOSE.xls", header = TRUE, sep="\t",row.names=1)

group_list <- factor(c(rep("Microglia", 6), rep("Microglia(PCDH9high)", 6)))

dat=log10(edgeR::cpm(pseudobulk_matrix)+1)
table(group_list)
n=t(scale(t(dat))) 
n <- na.omit(n)  

n[n>2]=2
n[n< -2]= -2
ac=data.frame(Celltype=group_list)
rownames(ac)=colnames(n) 
ac$Region<-c("Prefrontal cortex","Hippocampus","Hippocampus","Prefrontal cortex","Prefrontal cortex","Prefrontal cortex","Prefrontal cortex","Hippocampus","Hippocampus","Prefrontal cortex","Prefrontal cortex","Prefrontal cortex")
ac$Donor_status<-c("Healthy","Healthy","TLE","AD","RRMS","SPMS","Healthy","Healthy","TLE","AD","RRMS","SPMS")
library(dplyr)

ann_colors = list(Celltype = c(Microglia = "#F0E68B", `Microglia(PCDH9high)`= "#B2E61A"),
                  Region = c(Hippocampus = "#74CAFF", `Prefrontal cortex` = "#B3138A"),
                  Donor_status = c(Healthy = "#E68700", TLE="#9667BF",AD = "#5D6AB3", RRMS = "#52BEA5", SPMS = "#9BCA44"))


p1=pheatmap(n, 
         scale = "none",
         color =colorRampPalette(c("#3d4c9e","grey", "#eee814"))(100), show_rownames = T,show_colnames = F,cellwidth = 10, cellheight = 10,
         cluster_row = T, cluster_col = FALSE, annotation_col=ac, annotation_colors = ann_colors)


p1

n_transposed <- t(n)
p2 <- pheatmap(n_transposed, 
                       scale = "none",
                       color = colorRampPalette(c("#3d4c9e", "grey", "#eee814"))(100), 
                       show_rownames = FALSE,
                       show_colnames = TRUE,
                       cellwidth = 15,
                       cellheight = 15,
                       cluster_row = FALSE,
                       cluster_col = TRUE,
                       annotation_row = ac,
                       annotation_colors = ann_colors)
p2

ggsave(filename="extended_data_fig4g.pdf",plot=p2, width = 18, height = 7)